In [38]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy
import numpy as np
import seaborn as sns

Objetivos do Script:

Esse script tem como objetivo ler os dados que foram capturados da nossa amostra piloto e realizar alguns estudos em cima desses dados. 

Os 6 primeiros tópicos que eu separei abaixo são utilizados na grande maioria dos scripts de tratamento de dados desse estudo. 
Deixei padronizado para que seja possivel tratar simulações com um dispositivo ou vários.

Sumário:
  - Leitura dos dados
  - Separar publicações e confirmação de entrega por dispositivo simulado.
  - Criação de um dataframe com informações capturadas. 
  - Medidas de tendência centrar por dispositivo
  - Medidas de tendência centrar de todos os dispositivos.
  - Gráfico de dispersão da amostra.
  - Calculo do tamanho da amostra.

Como estamos trabalhando com uma amostra piloto temos como objetivo calcular o tamanho das amostras isso será realizado na ultima parte desse script.


In [39]:
data = pd.read_csv("./data.csv")
ports = data["Source Port"].unique()
ports = ports[ports !=1883]
data

,No.,Time,Source,Destination,Protocol,Length,Source Port,Message Identifier,Destination Port,Info
0,27,2.912368,192.168.15.8,130.185.238.170,MQTT,89,59595,NaN,1883,Connect Command
1,31,2.992240,192.168.15.8,130.185.238.170,MQTT,103,59595,"1,2",1883,"Subscribe Request (id=1) [devices-1], Publish ..."
2,32,2.992206,130.185.238.170,192.168.15.8,MQTT,70,1883,NaN,59595,Connect Ack
3,35,3.071512,130.185.238.170,192.168.15.8,MQTT,71,1883,1,59595,Subscribe Ack (id=1)
4,37,3.151560,130.185.238.170,192.168.15.8,MQTT,89,1883,2,59595,"Publish Message [devices-1], Publish Ack (id=2)"
...,...,...,...,...,...,...,...,...,...,...
3194,100821,1151.566650,130.185.238.170,192.168.15.8,MQTT,68,1883,NaN,32823,Ping Response
3195,100831,1152.473063,192.168.15.8,130.185.238.170,MQTT,68,59039,NaN,1883,Ping Request
3196,100832,1152.555783,130.185.238.170,192.168.15.8,MQTT,68,1883,NaN,59039,Ping Response
3197,100850,1153.459064,192.168.15.8,130.185.238.170,MQTT,68,44115,NaN,1883,Ping Request


In [40]:
#  - Separar publicações e confirmação de entrega por dispositivo simulado.
devices = {}
#  : devices será um dicionario que irá ter uma chave referente ao device e o valor será um dataframe referente aos respectivos devices 
for port in ports:  
  data = data.loc[ ~(data["Message Identifier"] == "1,2")]
  # data["Message Identifier"].dropna(replace=True)
  array_publish = data.loc[data["Source Port"] == port].dropna(subset=['Message Identifier'])
  array_publish_ack = data.loc[data["Destination Port"] == port].dropna(subset=['Message Identifier'])
  device = {"publish":array_publish, "publish_ack" : array_publish_ack}
  devices[port] = device

In [41]:
devicesNew = devices.copy()
for index in devices:
    if len(devices[index]["publish"]) <= 0:
        ports = ports.pop(index)



In [42]:
# - Criação de um dataframe com informações capturadas acima. 
data = {}
for port in ports:
    RTT = pd.DataFrame(columns=["Time_publish", "Time_ack"])
    for index, row in devices[port]["publish"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_publish"] = time_publish 

    for index, row in devices[port]["publish_ack"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_ack"] = time_publish
    RTT.loc[id, "Time_ack"] = time_publish
    RTT["RTT"]  =(RTT["Time_ack"] - RTT["Time_publish"] )* 1000
    data[port] = RTT


## Medidas de Tendência Central Por Device

In [43]:
i=1
data_mean = []
data_median = []
data_std = []
data_min = []
data_max = []

for port in ports:
  data_mean.append(data[port]["RTT"].mean())
  data_median.append(data[port]["RTT"].mean())
  data_std.append(data[port]["RTT"].mean())
  data_min.append(data[port]["RTT"].mean())
  data_max.append(data[port]["RTT"].mean())

  print("(Device-{})Mean Round Trip Time = {}".format(i ,data[port]["RTT"].mean()))
  print("(Device-{})Median Round Trip Time = {}".format(i ,data[port]["RTT"].median()))
  print("(Device-{})Standard deviation  Round Trip Time = {}".format(i ,data[port]["RTT"].std()))
  print("(Device-{})Min Round Trip Time = {}".format(i ,data[port]["RTT"].min()))
  print("(Device-{})Max Round Trip Time = {}".format(i ,data[port]["RTT"].max()))
  print("---------------------------------------------------------------")

  i+=1

(Device-1)Mean Round Trip Time = 158.6188952412027
(Device-1)Median Round Trip Time = 158.2952449999766
(Device-1)Standard deviation  Round Trip Time = 3.2214943171149177
(Device-1)Min Round Trip Time = 153.42493099990406
(Device-1)Max Round Trip Time = 199.7707710000043
---------------------------------------------------------------
(Device-2)Mean Round Trip Time = 153.2964506030165
(Device-2)Median Round Trip Time = 152.35144700000092
(Device-2)Standard deviation  Round Trip Time = 6.356644956146378
(Device-2)Min Round Trip Time = 150.24938999999904
(Device-2)Max Round Trip Time = 220.198607000043
---------------------------------------------------------------
(Device-3)Mean Round Trip Time = 162.30024654773754
(Device-3)Median Round Trip Time = 161.59279299999872
(Device-3)Standard deviation  Round Trip Time = 5.090533399446312
(Device-3)Min Round Trip Time = 159.34283699994012
(Device-3)Max Round Trip Time = 212.6043549999963
--------------------------------------------------------

# CÁLCULO DO TAMANHO DA AMOSTRA

In [44]:
nivel_de_significancia = 0.05

In [45]:
nivel_de_confiança = 1 - nivel_de_significancia
nivel_de_confiança

0.95

In [46]:
#Calculando o z com scipy
z = scipy.stats.norm.ppf(0.975)
z

1.959963984540054

Com nosso z calculado podemos seguir adiante

### $$z = 1.959963984540054 $$

Abaixo segue algumas formulas que iremos utilizar para calcular o numero de amostra



### Calculando o tamanho da nossa amostra

In [47]:
df_concatenado = pd.concat(data.values())
desvio_padrao_amostral = df_concatenado["RTT"].std()
media = df_concatenado["RTT"].mean()
e = 0.05 * media
n = (z * (desvio_padrao_amostral/e))**2
n

302.5185705299654

In [48]:
n = (z * (desvio_padrao_amostral/e))**2
n

302.5185705299654

In [49]:
desvio_padrao_amostral

70.81540983096086

In [51]:
ports

array([59595, 39995, 32823, 59039, 44115])

In [52]:
df_concatenado["RTT"][59595]

KeyError: 59595